In [ ]:
import os
import pickle

def get_mp4_files(directory):
    mp4_files = []
    for filename in os.listdir(directory):
        if filename.endswith('.mp4'):
            mp4_files.append(filename)
    return mp4_files

# Get the directory of the script
script_directory = os.path.dirname(__file__)

# Get a list of all .mp4 files in the script directory
mp4_files_list = get_mp4_files(script_directory)

# Save the list to a file
with open('mp4_files_list.pkl', 'wb') as file:
    pickle.dump(mp4_files_list, file)

print("List of .mp4 files:")
for mp4_file in mp4_files_list:
    print(mp4_file)

In [ ]:
import csv
import os
from scenedetect import detect, ContentDetector, split_video_ffmpeg
import pickle

def process_video(base_name):
    # Construct file and directory names
    video_file = f'{base_name}.mp4'
    output_directory = f'{base_name}_split_videos'
    csv_filename = os.path.join(output_directory, f'{base_name}_scene_detect.csv')

    os.makedirs(output_directory, exist_ok=True)

    def get_scene_info(scene):
        start_frame, start_timecode = int(scene[0].get_frames()), scene[0].get_timecode()
        end_frame, end_timecode = int(scene[1].get_frames()), scene[1].get_timecode()

        # Calculate lengths
        length_frames = end_frame - start_frame
        length_timecode_frames = int(scene[1].get_frames()) - int(scene[0].get_frames())
        length_timecode_seconds = scene[1].get_seconds() - scene[0].get_seconds()

        return start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, scene[0].get_seconds(), scene[1].get_seconds(), length_timecode_seconds

    with open(csv_filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Video Name', 'Scene Number', 'Start Frame', 'End Frame', 'Length (frames)', 'Start Timecode', 'End Timecode', 'Length (timecode)', 'Start Time (seconds)', 'End Time (seconds)', 'Length (seconds)', 'Cigarettes', 'Gun', 'Knife'])

        for i, scene in enumerate(detect(video_file, ContentDetector())):
            split_video_name = f'{base_name}-Scene-{i + 1:03d}.mp4'
            print(f'Splitting scene {i + 1}: {split_video_name}')
            split_video_ffmpeg(video_file, [scene], output_file_template=os.path.join(output_directory, split_video_name), show_progress=True)

            start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, start_seconds, end_seconds, length_timecode_seconds = get_scene_info(scene)
            
            # Add empty strings for Gun, Knife, Cigarettes columns
            csv_writer.writerow([split_video_name, i + 1, start_frame, end_frame, length_frames, start_timecode, end_timecode, length_timecode_frames, start_seconds, end_seconds, length_timecode_seconds, '', '', ''])

    print(f'Split videos and CSV are saved in the "{output_directory}" directory.')
    print(f'CSV file "{csv_filename}" has been created.')

# Read the list from the file
with open('mp4_files_list.pkl', 'rb') as file:
    mp4_files_list = pickle.load(file)

# Process each video
for base_name in mp4_files_list:
    process_video(os.path.splitext(base_name)[0])  # Remove the file extension


In [ ]:
import os

def get_mp4_files(directory):
    mp4_files = []
    for filename in os.listdir(directory):
        if filename.endswith('.mp4'):
            mp4_files.append(filename)
    return mp4_files

# Specify the directory where your .mp4 files are located
directory_path = '/path/to/your/directory'

# Get a list of all .mp4 files in the specified directory
mp4_files_list = get_mp4_files(directory_path)

# Print the list of .mp4 files
print("List of .mp4 files:")
for mp4_file in mp4_files_list:
    print(mp4_file)
